🧱 Cell 1: Import Required Libraries

In [1]:
import re
import json
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer, util




c:\Users\vidhi\Downloads\my_shl\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


 Cell 2: Configuration Parameters

In [2]:
# === Configuration ===
CSV_PATH    = "assessments_with_combined_text.csv"
EMBED_MODEL = "intfloat/e5-large-v2"
TOP_K       = 10
DESC_CAND_K = 30
BATCH_SIZE  = 64



Load Sentence Transformer Embedding Model and Assessment Recommendation Function

In [3]:
# === Load embedding model ===
model = SentenceTransformer(EMBED_MODEL,device='cpu')
def recommend_assessments(query, df, top_k=TOP_K, desc_cand_k=DESC_CAND_K):
    q_lower = query.lower()

    # --- Optional: Extract target duration from query ---
    m = re.search(r"(\d{1,3})\s*(?:minutes|min)", q_lower)
    dur_target = int(m.group(1)) if m else None

    # --- Stage 1: Semantic Filtering by title + description ---
    desc_title_texts = (df["cleaned_title"].fillna("") + " " + df["description"].fillna("")).tolist()
    desc_title_embs = model.encode(desc_title_texts, convert_to_tensor=True, batch_size=BATCH_SIZE)
    q_emb = model.encode(query, convert_to_tensor=True)
    desc_sims = util.cos_sim(q_emb, desc_title_embs)[0]
    top_desc_idx = torch.topk(desc_sims, k=min(len(df), desc_cand_k)).indices.tolist()
    sem_filtered_df = df.iloc[top_desc_idx].copy()
    sem_filtered_df["desc_title_sim"] = [desc_sims[i].item() for i in top_desc_idx]

    # --- Stage 2: Keyword Filtering ---
    query_tokens = set(re.findall(r'\w+', q_lower))
    def keyword_match(kw):
        if pd.isna(kw): return False
        kw_tokens = set(re.findall(r'\w+', kw.lower()))
        return not query_tokens.isdisjoint(kw_tokens)

    keyword_filtered_df = df[df["keywords"].apply(keyword_match)]

    # --- Intersect Stage 1 and 2 ---
    common_ids = set(sem_filtered_df.index).intersection(set(keyword_filtered_df.index))
    if not common_ids:
        return []  # fallback: return empty if no match
    final_df = df.loc[sorted(common_ids)].copy()

    # --- Final Ranking using combined column ---
    combined_embs = model.encode(final_df["combined_text"].fillna("").tolist(), convert_to_tensor=True, batch_size=BATCH_SIZE)
    combined_sims = util.cos_sim(q_emb, combined_embs)[0]
    final_df["combined_sim"] = [combined_sims[i].item() for i in range(len(final_df))]

    # --- Duration Score ---
    dur_scores = []
    for _, row in final_df.iterrows():
        if dur_target is not None and pd.notna(row["duration_minutes"]):
            diff = abs(int(row["duration_minutes"]) - dur_target)
            score = max(0.0, 1 - diff / max(1, dur_target))
        else:
            score = 0.5
        dur_scores.append(score)
    final_df["dur_score"] = dur_scores

    # --- Final Score and Sort ---
    final_df["final_score"] = list(zip(final_df["dur_score"], final_df["combined_sim"]))
    final_df_sorted = final_df.sort_values("final_score", ascending=False)

    return final_df_sorted["url"].head(top_k).tolist()



Define Evaluation Metrics, Test Queries for Evaluation and Run Evaluation on Test Set

In [4]:
# === Metrics ===
def recall_at_k(pred, relevant, k):
    return len(set(pred[:k]) & set(relevant)) / len(relevant) if relevant else 0.0

def ap_at_k(pred, relevant, k):
    hits = 0
    score = 0.0
    for i, p in enumerate(pred[:k], start=1):
        if p in relevant:
            hits += 1
            score += hits / i
    return score / min(len(relevant), k) if relevant else 0.0

# === Test Set ===
test_queries = [
    {
        "query": "I am hiring for Java developers who can also collaborate effectively with my business teams. Looking for an assessment(s) that can be completed in 40 minutes.",
        "relevant": [
            "https://www.shl.com/products/product-catalog/view/core-java-entry-level-new/",
            "https://www.shl.com/products/product-catalog/view/java-8-new/",
            "https://www.shl.com/products/product-catalog/view/core-java-advanced-level-new/",
            "https://www.shl.com/products/product-catalog/view/agile-software-development/",
            "https://www.shl.com/products/product-catalog/view/technology-professional-8-0-job-focused-assessment/",
            "https://www.shl.com/products/product-catalog/view/computer-science-new/"
        ]
    },
    {
        "query": "I want to hire new graduates for a sales role in my company, the budget is for about an hour for each test. Give me some options",
        "relevant": [
            "https://www.shl.com/products/product-catalog/view/entry-level-sales-7-1/",
            "https://www.shl.com/products/product-catalog/view/entry-level-sales-sift-out-7-1/",
            "https://www.shl.com/products/product-catalog/view/entry-level-sales-solution/",
            "https://www.shl.com/products/product-catalog/view/sales-representative-solution/",
            "https://www.shl.com/products/product-catalog/view/sales-support-specialist-solution/",
            "https://www.shl.com/products/product-catalog/view/technical-sales-associate-solution/",
            "https://www.shl.com/products/product-catalog/view/svar-spoken-english-indian-accent-new/",
            "https://www.shl.com/products/product-catalog/view/sales-and-service-phone-solution/",
            "https://www.shl.com/products/product-catalog/view/sales-and-service-phone-simulation/",
            "https://www.shl.com/products/product-catalog/view/english-comprehension-new/"
        ]
    },
    {
        "query": "Content Writer required, expert in English and SEO.",
        "relevant": [
            "https://www.shl.com/products/product-catalog/view/drupal-new/",
            "https://www.shl.com/products/product-catalog/view/search-engine-optimization-new/",
            "https://www.shl.com/products/product-catalog/view/administrative-professional-short-form/",
            "https://www.shl.com/products/product-catalog/view/entry-level-sales-sift-out-7-1/",
            "https://www.shl.com/products/product-catalog/view/general-entry-level-data-entry-7-0-solution/"
        ]
    },
    
    {
        "query": "ICICI Bank Assistant Admin, Experience required 0-2 years, test should be 30-40 mins long",
        "relevant": [
            "https://www.shl.com/products/product-catalog/view/administrative-professional-short-form/",
            "https://www.shl.com/products/product-catalog/view/verify-numerical-ability/",
            "https://www.shl.com/products/product-catalog/view/financial-professional-short-form/",
            "https://www.shl.com/products/product-catalog/view/bank-administrative-assistant-short-form/",
            "https://www.shl.com/products/product-catalog/view/general-entry-level-data-entry-7-0-solution/",
            "https://www.shl.com/products/product-catalog/view/basic-computer-literacy-windows-10-new/"
        ]
    }
]

# === Run Evaluation ===
df = pd.read_csv(CSV_PATH)
recalls, aps = [], []
print(f"\nEvaluating on {len(test_queries)} queries...\n")
for i, entry in enumerate(test_queries, 1):
    preds = recommend_assessments(entry["query"], df)
    r = recall_at_k(preds, entry["relevant"], TOP_K)
    a = ap_at_k(preds, entry["relevant"], TOP_K)
    recalls.append(r); aps.append(a)
    print(f"Query {i}: Recall@{TOP_K} = {r:.3f},  AP@{TOP_K} = {a:.3f}")

print(f"\nMean Recall@{TOP_K} = {sum(recalls)/len(recalls):.3f}")
print(f" Mean AP@{TOP_K}     = {sum(aps)/len(aps):.3f}")



Evaluating on 4 queries...

Query 1: Recall@10 = 0.500,  AP@10 = 0.183
Query 2: Recall@10 = 0.400,  AP@10 = 0.315
Query 3: Recall@10 = 0.200,  AP@10 = 0.200
Query 4: Recall@10 = 0.167,  AP@10 = 0.167

Mean Recall@10 = 0.317
 Mean AP@10     = 0.216
